# From Here I am Starting BIG QUERY Project FOR Library Management

---



>
SO first I have uploaded some of Data related to it in Goggle Big Query and created tables like database and here i will write some sql queries to perform some the operations


In [3]:
from google.cloud import bigquery

In [4]:
key_path = '/content/uber-data-387518-d26ab336faa3.json'
project_id = 'uber-data-387518'
client = bigquery.Client.from_service_account_json(key_path, project=project_id)

 1- How many copies of the book titled "The Lost Tribe" are owned by the library branch whose name is "Sharpstown"?

In [5]:


SQL_Q="""
  SELECT copies.book_copies_BranchID, branch.library_branch_BranchName,
	   copies.book_copies_No_Of_Copies,
	   book.book_Title
	   FROM `Library_managment.book_copies` AS copies
			INNER JOIN `Library_managment.Book` AS book ON copies.book_copies_BookID = book.book_ID
			INNER JOIN `Library_managment.Library` AS branch ON book_copies_BranchID = branch.library_id
	   WHERE book.book_Title='The Lost Tribe' AND branch.library_branch_BranchName='Sharpstown' 
     """
res=client.query(SQL_Q)


for row in res.result():
  print(row.values())


(1, 'Sharpstown', 5, 'The Lost Tribe')


2- How many copies of the book titled "The Lost Tribe" are owned by each library branch?

In [6]:
SQL_Q="""

     SELECT copies.book_copies_BranchID, branch.library_branch_BranchName,
	   copies.book_copies_No_Of_Copies,
	   book.book_Title
	   FROM `Library_managment.book_copies` AS copies
			INNER JOIN `Library_managment.Book` AS book ON copies.book_copies_BookID = book.book_ID
			INNER JOIN `Library_managment.Library` AS branch ON book_copies_BranchID = branch.library_id
	   WHERE book.book_Title = 'The Lost Tribe' 
     """

res=client.query(SQL_Q)


for row in res.result():
  print(row.values())


(1, 'Sharpstown', 5, 'The Lost Tribe')
(2, 'Central', 5, 'The Lost Tribe')
(3, 'Saline', 5, 'The Lost Tribe')
(4, 'Ann Arbor', 5, 'The Lost Tribe')


3- Retrieve the names of all borrowers who do not have any books checked out.

In [7]:
SQL_Q= """
SELECT borrower_BorrowerName FROM `Library_managment.Borrower`
	WHERE NOT EXISTS
		(SELECT * FROM `Library_managment.book_loans`
		WHERE book_loans_CardNo = borrower_CardNo_)
    """

res=client.query(SQL_Q)

for row in res.result():
  print(row.values())


('Michael Horford',)
('Joe Smith',)


4- For each library branch, retrieve the branch name and the total number of books loaned out from that branch.

> 

In [8]:
SQL_Q="""
SELECT  Branch.library_branch_BranchName, COUNT (Loans.book_loans_BranchID)
		FROM `Library_managment.book_loans` AS Loans
			INNER JOIN `Library_managment.Library` AS Branch ON Loans.book_loans_BranchID = Branch.library_id
			GROUP BY library_branch_BranchName
"""
res=client.query(SQL_Q)

for row in res.result():
  print(row.values())



('Ann Arbor', 20)
('Central', 11)
('Saline', 10)
('Sharpstown', 10)


5- Retrieve the names, addresses, and number of books checked out for all borrowers who have more than five books checked out.

In [9]:
SQL_Q="""

SELECT Borrower.borrower_BorrowerName, Borrower.borrower_BorrowerAddress,
		   COUNT(Borrower.borrower_BorrowerName)
		   FROM `Library_managment.book_loans` As Loans
		   			INNER JOIN `Library_managment.Borrower` AS Borrower ON Loans.book_loans_CardNo = Borrower.borrower_CardNo_
					GROUP BY Borrower.borrower_BorrowerName, Borrower.borrower_BorrowerAddress
		   HAVING COUNT(Borrower.borrower_BorrowerName) >=5
"""

res=client.query(SQL_Q)

for row in res.result():
  print(row.values())


('Jane Smith', '1321 4th Street, New York, NY 10014', 14)
('Harry Emnace', '121 Park Drive, Ann Arbor, MI 48104', 6)
('Haley Jackson', '231 52nd Avenue New York, NY 10014', 8)
('Tom Li', '981 Main Street, Ann Arbor, MI 48104', 11)


6- For each book authored by "Stephen King", retrieve the title and the 
number of copies owned by the library branch whose name is "Central"

In [10]:
SQL_Q="""
SELECT Branch.library_branch_BranchName, Book.book_Title, Copies.book_copies_No_Of_Copies
		   FROM `Library_managment.Book_Authors` AS Authors
				INNER JOIN `Library_managment.Book` AS Book ON Authors.book_authors_BookID = Book.book_ID
				INNER JOIN `Library_managment.book_copies` AS Copies ON Authors.book_authors_BookID = Copies.book_copies_BookID
				INNER JOIN `Library_managment.Library` AS Branch ON Copies.book_copies_BranchID = Branch.library_id
			WHERE Branch.library_branch_BranchName = 'Central' AND Authors.book_authors_AuthorName ='Stephen King'
    """
  
res=client.query(SQL_Q)

for row in res.result():
  print(row.values())



('Central', 'It', 5)
('Central', 'The Green Mile', 5)
